In [2]:
# Import pandas, numpy, matplotlib, seaborn libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime
import time

import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics

In [3]:
#Load the 'trips' dataset
dftrips_copy = pd.read_csv('trips_copy.csv')
dftrips_copy

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNED_TRIPTIME(s),ACTUAL_TRIPTIME(s),HOUROFDAY,DAYOFWEEK,MONTHOFYEAR
0,2018-02-07,6253783,68,68_80,1,87245,84600,87524.0,84600.0,2645,2924,23.0,2,2
1,2018-02-07,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,4057,4057,7.0,2,2
2,2018-02-07,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,3412,4247,9.0,2,2
3,2018-02-07,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,2841,4020,15.0,2,2
4,2018-02-07,6253175,14,14_15,1,85383,81600,84682.0,81608.0,3783,3074,23.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182529,2018-05-14,6765849,123,123_36,2,61560,57840,61365.0,57859.0,3720,3506,16.0,0,5
2182530,2018-05-14,6765469,75,75_17,1,53416,48600,NaN,48823.0,4816,4816,14.0,0,5
2182531,2018-05-14,6765486,33D,33D_62,2,29460,26400,29904.0,NaN,3060,3060,7.0,0,5
2182532,2018-05-14,6764987,70,70_60,1,65277,60600,66341.0,NaN,4677,4677,17.0,0,5


In [4]:
dftrips_copy.dtypes

DAYOFSERVICE            object
TRIPID                   int64
LINEID                  object
ROUTEID                 object
DIRECTION                int64
PLANNEDTIME_ARR          int64
PLANNEDTIME_DEP          int64
ACTUALTIME_ARR         float64
ACTUALTIME_DEP         float64
PLANNED_TRIPTIME(s)      int64
ACTUAL_TRIPTIME(s)       int64
HOUROFDAY              float64
DAYOFWEEK                int64
MONTHOFYEAR              int64
dtype: object

In [5]:
#save all the bus line in a list
lineid_list = dftrips_copy['LINEID'].unique()
lineid_list

array(['68', '25B', '45A', '25A', '14', '77A', '39', '16', '40D', '27B',
       '142', '83', '130', '15', '46A', '33', '7', '39A', '49', '1',
       '123', '41', '67X', '59', '9', '40', '239', '76', '84', '53',
       '185', '151', '13', '15B', '65B', '29A', '61', '140', '79A', '38A',
       '31', '33B', '69', '44', '42', '67', '184', '238', '145', '17A',
       '32', '27A', '17', '27X', '18', '122', '54A', '66', '150', '56A',
       '37', '27', '15A', '65', '11', '47', '79', '83A', '63', '4', '120',
       '41C', '70', '84A', '220', '39X', '32X', '68A', '84X', '38', '102',
       '270', '51X', '33X', '75', '26', '66A', '31A', '111', '14C', '114',
       '76A', '44B', '161', '7A', '43', '25', '104', '33A', '16C', '42D',
       '31B', '66X', '31D', '33D', '41B', '40B', '7D', '46E', '38D',
       '118', '51D', '15D', '41A', '25D', '66B', '38B', '236', '7B',
       '41X', '69X', '68X', '25X', '40E', '70D', '116', '77X', '16D',
       '33E', '41D'], dtype=object)

In [6]:
count = 1

#All the routes below are the routes we identified and decided to use 2018 historical data for modeling.
#lineid_list = ['77A', '16', '40D', '142', '83', '46A', '49', '9', '40', '151', '13', '15B', '61', '140', '79A', '38A', '69', '54A', '150', '150', '56A', '120', '41C', '39X', '68A', '84X', '38', '33X', '26', '44B', '42D', '33D', '40B', '46E', '118', '51D', '15D', '38B', '7B', '41X', '69X', '70D', '77X']
#lineid_list = ['102','104','111','114','161','17','184','185','18','220','236','238','270','59']
#lineid_list = ['63','75','76','11','122','123','130','145','15A','15','1','27A','27B','27','27X','32X','33','37','38D','39A','39']
#lineid_list = ['41B','41','42','43','47','53','65','68','70','79','7A','7','7D','83A','84A','84','14']
lineid_list = ['4','44','16D','33E','40E']

start_time = time.time()

# looping over each lineid in the list to apply the data cleaning and model establishing processes we designed.
for lineid in lineid_list:
    
    now = time.time()
    timer = now - start_time
    print("Now working on line:", lineid, "number", count, "in the sequence. Time since start", timer)
    count += 1
    data = lineid
    #we pass the info of lineid to the encapsulated notebook file. 
    %store data
    del data
    %run ./Historical_Data_Processing_Child.ipynb

Now working on line: 4 number 1 in the sequence. Time since start 0.00017905235290527344
Stored 'data' (str)


<ipython-input-6-77b82dba6fe9>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat["ACTUALTIME_DEP_y"][i] = df_concat["PLANNEDTIME_DEP_y"][i]
<ipython-input-6-77b82dba6fe9>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['DAYOFSERVICE'][i] = df_concat['DAYOFSERVICE'][i] + pd.Timedelta(hours=df_concat['HOUROFDAY'][i])
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


Now working on line: 44 number 2 in the sequence. Time since start 364.13896894454956
Stored 'data' (str)


<ipython-input-6-77b82dba6fe9>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat["ACTUALTIME_DEP_y"][i] = df_concat["PLANNEDTIME_DEP_y"][i]
<ipython-input-6-77b82dba6fe9>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['DAYOFSERVICE'][i] = df_concat['DAYOFSERVICE'][i] + pd.Timedelta(hours=df_concat['HOUROFDAY'][i])
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


Now working on line: 16D number 3 in the sequence. Time since start 563.1498599052429
Stored 'data' (str)


<ipython-input-6-77b82dba6fe9>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat["ACTUALTIME_DEP_y"][i] = df_concat["PLANNEDTIME_DEP_y"][i]
<ipython-input-6-77b82dba6fe9>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['DAYOFSERVICE'][i] = df_concat['DAYOFSERVICE'][i] + pd.Timedelta(hours=df_concat['HOUROFDAY'][i])
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


Now working on line: 33E number 4 in the sequence. Time since start 664.8414940834045
Stored 'data' (str)


<ipython-input-6-77b82dba6fe9>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['DAYOFSERVICE'][i] = df_concat['DAYOFSERVICE'][i] + pd.Timedelta(hours=df_concat['HOUROFDAY'][i])
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


Now working on line: 40E number 5 in the sequence. Time since start 764.4497628211975
Stored 'data' (str)


<ipython-input-6-77b82dba6fe9>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat["ACTUALTIME_DEP_y"][i] = df_concat["PLANNEDTIME_DEP_y"][i]
<ipython-input-6-77b82dba6fe9>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat['DAYOFSERVICE'][i] = df_concat['DAYOFSERVICE'][i] + pd.Timedelta(hours=df_concat['HOUROFDAY'][i])
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [9]:
X_test_25 = pd.read_csv('X_test_trial.csv')
X_test_25

,DIRECTION,PROGRNUMBER,STOPPOINTID,PLANNED_TRIPTIME(s),HOUROFDAY,DAYOFWEEK,MONTHOFYEAR,temp,wind_speed,rain_1h,wind_gust
0,1,25,7239,3317,18.0,3,2,3.00,9.3,0.00,0.0
1,2,21,2245,2303,22.0,5,4,12.39,2.6,0.00,0.0
2,2,30,1473,2805,9.0,3,8,15.39,5.1,0.00,0.0
3,1,30,2216,2559,13.0,3,12,6.39,8.8,0.11,14.4
4,1,15,2191,2595,10.0,4,10,12.92,5.7,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
129293,2,30,1473,1780,22.0,0,8,17.39,2.6,0.00,0.0
129294,1,15,2191,2764,8.0,3,7,15.46,4.1,0.00,0.0
129295,2,33,7453,2078,19.0,1,10,13.92,6.7,0.00,0.0
129296,2,11,2235,3606,7.0,2,11,14.39,8.2,0.37,0.0


In [ ]:
#Here we test loading the pickle file 

In [25]:
dictionary = {'DIRECTION': 1, 'PROGRNUMBER': 60, 'PLANNED_TRIPTIME(s)': 4289.571428571428, 'HOUROFDAY': 16, 'DAYOFWEEK': 4, 'MONTHOFYEAR': 7, 'temp': 297.87, 'wind_speed': 5.14, 'rain_1h': 0}

In [26]:
var = pd.DataFrame([dictionary])
var

,DIRECTION,PROGRNUMBER,PLANNED_TRIPTIME(s),HOUROFDAY,DAYOFWEEK,MONTHOFYEAR,temp,wind_speed,rain_1h
0,1,60,4289.571429,16,4,7,297.87,5.14,0


In [27]:
#Load the model
line25_model = pickle.load(open('ModelResult_LinearRegression/PickleFiles_11.pkl', 'rb'))

In [28]:
line25_model.predict(var)

array([4500.05912285])